### BERT Score로 입력문을 요약하는 모델이 생성한 요약문이 입력한 문장을 얼마나 잘 요약하는지 평가해보기
- 기존의 ROUGE 평가와 다르게 단순히 입력문과 생성문의 단어 매칭을 확인하는 것이 아니라 인코더로 입력 문장을 이해하는 Bert 모델을 활용해 두 문장이 의미적으로 유사한지 평가
- ex) "강아지가 뛰어놀고 있다" <-> "개의 활동성이 높다" 의 두 문장은 일치하는 단어는 없지만 의미적으로 유사하면 높은 점수 부여
- 평가 데이터 셋 : 한국어 기사 본문을 요약한 'daekeun-ml/naver-news-summarization-ko'의 test 데이터셋 중 100개를 추출하여 사용

In [35]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text2text-generation", max_length=128, model="edgeun/t5-small-korean-news-summarizer")

Device set to use cuda:0


In [36]:
inputs = "summarize: " + """
달력은 봄에 가까워지고 있지만, 아직 두터운 겨울 외투 없이는 바깥에 나서기가 어렵습니다.
막바지 강추위는 이번 주말까지 이어지겠는데요.
서울의 아침 기온이 영하 7도 선을 유지하면서 평년 기온을 4도 가량 밑돌겠습니다.
특히, 한파특보가 내려진 일부 중부와 경북 지역은 영하 10도 안팎까지 내려가겠고, 전국에 바람이 강하게 불며 체감 온도를 뚝뚝 떨어트리겠습니다.
맑은 날씨가 이어지면서 건조특보 지역은 나날이 확대되고 있는데요.
동해안을 따라서는 건조경보가, 서울 등 중부와 동쪽 많은 지역에는 건조주의보가 내려졌습니다. 꺼진 불씨도 다시 한번 확인하시기 바랍니다.
아침 기온은 서울 영하 7도, 청주와 대구가 영하 5도로 오늘만큼 춥겠고요.
낮에도 서울은 2도, 광주는 3도에 머물겠습니다.
일요일에는 충남과 호남, 제주에 비나 눈이 내리겠고, 다음 주부터는 서서히 봄기운이 느껴지겠습니다.
"""

result = pipe(inputs)

In [17]:
result[0]['generated_text']

'서울의 아침 기온이 영하 7도 선을 유지하면서 평년 기온을 4도 가량 밑돌겠고, 낮에도 서울은 2도, 광주는 3도에 머물겠고, 전국에 바람이 강하게 불며 체감 온도를 뚝뚝 떨어트리며, 낮에도 서울은 2도, 광주는 3도에 머물겠습니다.'

In [1]:
# !pip install datasets -qqq
from datasets import load_dataset
test_dataset = load_dataset("daekeun-ml/naver-news-summarization-ko", split='test').shuffle(seed=42).select(range(100))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.0 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/787 [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/66.3M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/7.45M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/8.17M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/22194 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2466 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2740 [00:00<?, ? examples/s]

In [2]:
test_dataset

Dataset({
    features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
    num_rows: 100
})

In [ ]:
from huggingface_hub import login

login(token="****")

In [5]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_id = "edgeun/t5-small-korean-news-summarizer"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.37M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/788 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/383M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [6]:
# 모델이 지금 어디에 있는지 확인
model.device  # device(type='cpu') -> cpu에 모델이 있음

device(type='cpu')

In [7]:
# 모델 GPU로 보내기
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(50358, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(50358, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

In [11]:
from torch.utils.data import DataLoader

# DataLoader를 사용해서 배치 단위로 데이터 가져오기
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [13]:
from tqdm import tqdm

predictions = []
references = []

for batch in tqdm(test_dataloader):
    input_texts = ["summarize: " + doc for doc in batch["document"]]
    reference_texts = batch["summary"]

    # 배치 토큰화
    inputs = tokenizer(input_texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {key: val.to(device) for key, val in inputs.items()}  # 입력 텐서를 GPU로 이동

    # 모델 추론 (배치 처리)
    output_ids = model.generate(**inputs, do_sample=True, max_length=128, num_beams=4, no_repeat_ngram_size=2)

    # 배치 디코딩
    batch_predictions = tokenizer.batch_decode(output_ids, skip_special_tokens=True)

    # 결과 저장
    predictions.extend(batch_predictions)
    references.extend(reference_texts)

100%|██████████| 13/13 [00:46<00:00,  3.60s/it]


In [19]:
predictions[:5]

['지금 환율이 1300원을 예사로 넘으면서 여기저기서 우려의 목소리가 나옵니다.',
 '금융권의 클라우드 도입이 궤도에 오르고 있는 가운데 안정적인 IT인프라 운영을 위해 금융사들은 온프레미스 퍼블릭 프라이빗 클라우디아를 혼재해 사용하는 ‘플러그 인 스마트 워크 스테이션 서비스 제공 혁신 방안 과 주제로 경험 공유할 계획이다.',
 '3일 한국연구재단은 대구경북과학기술원 DGIST 이주혁 교수 연구팀과 협력해 투명하고 신축성 있는 고분자 겔 단일층으로 구성된 마찰전기 나노발전기를 개발, 표면 전극 유전소재 등의 버려지는 에너지를 수확 전기에너지로 변환할 수 없는 소자로써 전원 공급 장치로의 활용 가능성에 대해서도 주목을 받고 있다.',
 '산업통상자원부와 한국전력에 따르면 전기요금과 가스요금이 1일 함께 오르면서 물가 부담은 더 커질 전망이다.',
 '정부는 1일 정부서울청사에서 박두선 기획재정부 2차관 주재로 제2차 비상경제차관회의를 열고 물가 상승기에 편승한 지역별 주유소 가격 담합 가짜석유 유통과 같은 불법행위를, 디플레이션 압력이 계속 이어질 것이라는 전망을 내놨다.']

In [22]:
# !pip install bert_score -qqq
from bert_score import BERTScorer

# BERTScore 초기화
scorer = BERTScorer(model_type="klue/roberta-large", num_layers=24, batch_size=8, lang="ko", rescale_with_baseline=False, device=device)  # "klue/roberta-large": 한국어 모델

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
# Bert Score로 정밀도, 재현율, F1 스코어 계산
P, R, F1 = scorer.score(predictions, references)

In [82]:
P

tensor([0.7679, 0.7607, 0.8684, 0.8264, 0.7767, 0.8463, 0.7657, 0.6847, 0.6293,
        0.8896, 0.8932, 0.7561, 0.9012, 0.9229, 0.8974, 0.7610, 0.9207, 0.7146,
        0.9375, 0.7915, 0.8431, 0.8051, 0.9665, 0.6672, 0.6796, 0.7305, 0.8593,
        0.9455, 0.8171, 0.9583, 0.8855, 0.7916, 0.8019, 0.8944, 0.8127, 0.8223,
        0.7083, 0.9909, 0.9708, 0.8696, 0.9350, 0.9640, 0.8851, 0.9467, 0.9250,
        0.9512, 0.8677, 0.8639, 0.8970, 0.8021, 0.8596, 0.7509, 0.9328, 0.7580,
        0.7587, 0.8704, 0.7588, 0.8121, 0.7307, 0.7772, 0.9206, 0.9017, 0.7465,
        0.9708, 0.8313, 0.7617, 0.7647, 0.9227, 0.9049, 0.8070, 0.9366, 0.8112,
        0.7851, 0.8965, 0.6785, 0.8789, 0.6995, 0.9517, 0.9008, 0.9508, 0.8340,
        0.9144, 0.7774, 0.7406, 0.7995, 0.7333, 0.9293, 0.9104, 0.8597, 0.8151,
        0.8167, 0.9180, 0.8426, 0.8128, 0.9023, 0.9445, 0.8283, 0.8482, 0.8347,
        0.8820])

In [92]:
import pandas as pd

result = pd.DataFrame({
        '생성한 요약문' : references,
        '정밀도' : P.tolist(),
        '재현율' : R.tolist(),
        'F1' : F1.tolist() })

result.sort_values(by='F1', ascending=False)

,생성한 요약문,정밀도,재현율,F1
22,3일 정부서울청사에서 추경호 경제부총리 겸 기획재정부 장관이 정부서울청사에서 열린 ...,0.966539,0.958942,0.962725
37,롯데바이오로직스는 글로벌 바이오 기업들과의 협업을 강화하며 바이오 사업 추진을 가속...,0.990880,0.916545,0.952264
38,6일 오후 부산상공회의소 대강당에서 동남권관문공항추진위원회와 신공항추진범시민운동본부...,0.970841,0.909372,0.939101
95,8계의 노벨상이라 불리며 수학계의 노벨상 으로 불리는 필즈상을 수상한 허준이 미국 ...,0.944504,0.889136,0.915984
78,한국우편사업진흥원은 4일부터 17일까지 14일간 여름 제철 과일과 장어 삼계탕 등 ...,0.900795,0.927763,0.914081
...,...,...,...,...
7,일다의 브랜딩 팀은 이 프로젝트를 위해 지난 10개월간 카메라를 달고 서울 도심을 ...,0.684707,0.679947,0.682319
23,대신증권은 8일 에스티팜의 올 2분기 실적이 시장 예상치 컨센서스를 소폭 밑돌 것이...,0.667176,0.607715,0.636059
36,부산주에 본사를 둔 금융 공기업의 임원 중 일부가 이미 임기를 끝냈으나 후임 인선 ...,0.708313,0.564580,0.628332
8,뉴욕증시가 하반기 첫 거래날 장중 큰 변동성을 보이다가 결국 상승 마감했는데 이날 ...,0.629291,0.604076,0.616426


In [93]:
print(f"평균 정밀도: {result['정밀도'].mean():.4}")
print(f"평균 재현율: {result['재현율'].mean():.4}")
print(f"평균 F1 스코어: {result['F1'].mean():.4}")

평균 정밀도: 0.8414
평균 재현율: 0.7905
평균 F1 스코어: 0.8125
